In [1]:
from ml_pipeline import experiment

/home/rutger/miniconda3/envs/pynlp/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
2022-09-23 10:37:21.135432: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-09-23 10:37:21.135466: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
res = experiment.run(
    task_name='vua_format',
    data_dir='../pynlp/data/dataset1/',
    pipeline_name='naive_bayes_counts',
    print_predictions=False
)

>> Running vua_format experiment
>> Loading data...
>> retrieving train/data instances...
>> training pipeline naive_bayes_counts
>> testing...
>> evaluation...
              precision    recall  f1-score   support

         CAG       0.30      0.43      0.35       555
         NAG       0.70      0.52      0.60      1113
         NOT       0.46      0.93      0.61       620
         OAG       0.43      0.29      0.34       505
         OFF       0.34      0.31      0.33       240
        hate       0.80      0.10      0.18       239
      noHate       0.36      0.03      0.06       239

    accuracy                           0.47      3511
   macro avg       0.48      0.37      0.35      3511
weighted avg       0.51      0.47      0.44      3511



In [3]:
import pandas as pd

In [4]:
df_gibert_test = pd.read_csv('../pynlp/data/gibert/testData.csv', delimiter='\t')

In [5]:
res

Pipeline(steps=[('prep',
                 <ml_pipeline.preprocessing.Preprocessor object at 0x7f4fa3070cd0>),
                ('frm', CountVectorizer()), ('clf', MultinomialNB())])

In [6]:
prep = res['prep']
frm = res['frm']
clf = res['clf']

In [7]:
res = prep.transform(df_gibert_test['Text'])
res = frm.transform(res)
pred = clf.predict(res)

In [9]:
df_gibert_test['Pred'] = pred

In [12]:
df_gibert_test.head(5)

,Id,Text,Label,Pred
0,30399453_2,I can give you her Skype and she will answer a...,noHate,NOT
1,13456898_1,"I know exactly what you 're saying , I just ha...",hate,OFF
2,13605248_8,"Norway : 3.3 ( 2001 ) , 3.9 ( 2002 ) , 3.5 ( 2...",noHate,hate
3,14101084_1,I used to enter horse competitions until I was...,noHate,NAG
4,12849464_4,And the sad thing is the white students at tho...,hate,NOT


**TODO: Deal with overlapping labels**

Need to decide on a single annotation scheme, or modify the training objective somehow.

Right now, a model is punished for classifying a message as e.g. `NOT` when the corresponding label is `noHate`.
Both of these point at the same thing, namely that the text does not contain any hate speech / offensive language.